In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import qiskit as qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, transpile, Aer, IBMQ

import ramsey_experiment

# Loading your IBM Quantum account(s)
#IBMQ.save_account(os.environ.get('IBM_KEY'), overwrite=True)
#IBMQ.load_account()
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='4cb4dd43af6af16cc340156cf1dfebb32ec9f233cec56a0c6e98e9a574f99d96c7a4867d4541dc9394c58f81efe019ee4feb8fa17b475f79952b0f66bf000f4c'
)

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='<IBM Quantum API key>')


In [11]:
n = 4
circuits = []
# backend = service.least_busy(operational=True, simulator=False)
backend = service.get_backend("ibm_kyoto")

delay = np.arange(0, 40, 1)
for d in delay:
    circuit = QuantumCircuit(n)
    circuit.h(0)
    circuit.barrier()
    circuit.delay(int(d),0, 'us')
    circuit.barrier()
    circuit.h(0)
    circuit.measure_all()
    transpiled_circ = transpile(circuit, backend, scheduling_method='alap')
    circuits.append(transpiled_circ)
#circuits[100].draw('mpl')

In [8]:
from qiskit.quantum_info import Pauli

ZIII = Pauli('ZIII')
IZII = Pauli('IZII')
IIZI = Pauli('IIZI')
IIIZ = Pauli('IIIZ')
Z = 'Z'+'I'*(126)
Z = Pauli(Z)

In [ ]:
from qiskit.tools import job_monitor

service = QiskitRuntimeService(channel="ibm_quantum")
# backend = service.least_busy(operational=True, simulator=False)
#backend = service.get_backend("ibmq_qasm_simulator")
options = Options()
options.resilience_level = 0
options.optimization_level = 0
#use only qubit number 8
initial_layout = [108]
estimator = Estimator(backend, options=options)
job = estimator.run(circuits=circuits, observables=[Z]*40, shots = 8000, initial_layout=initial_layout)
#job = backend.run(circuits,400)
job_monitor(job)

#job = execute(circuits, backend, shots=4321)
#job.result()

Job Status: job is queued (None)

KeyboardInterrupt: 

In [ ]:
plt.xlim([0, 3000])
plt.plot(delay, job.result().values)

In [ ]:
from scipy.signal import find_peaks

T = 3000
measurements = 300
a = job.result().values
b = a[::-1]
extended = np.concatenate((a,b))
fft_output_ext = np.fft.fft(extended)

import numpy as np
import matplotlib.pyplot as plt

# Set the sampling rate based on measurements
sample_rate = measurements / T

# Compute the positive frequencies for the FFT output
frequencies_ext = np.fft.fftfreq(2 * len(job.result().values), 1 / sample_rate)
frequencies_ext*=(2 * np.pi)
positive_indices = np.where(frequencies_ext > 0)
positive_magnitudes = np.abs(fft_output_ext)[positive_indices]


frequencies  = frequencies_ext[positive_indices]
weights =  np.abs(fft_output_ext)[positive_indices]

# Calculating cumulative weights
cumulative_weights = np.cumsum(weights)

# Finding the median position
total_weight = np.sum(weights)
half_total_weight = total_weight / 2

median_index = np.where(cumulative_weights >= half_total_weight)[0][0]
weighted_median_frequency = frequencies[median_index]

print(f'Weighted median frequency: {weighted_median_frequency}')
# Plot and annotate results
plt.plot(frequencies_ext, np.abs(fft_output_ext))
#plt.axvline(x=W[k], color='orange', linestyle='--',label=f'W{k} = {W[k]}')
plt.axvline(x=weighted_median_frequency, color='green', linestyle='--',label=f'W_found{weighted_median_frequency} ')

#a = L[k]

## find peaks ##
peaks, _ = find_peaks(positive_magnitudes)
peak_magnitudes = positive_magnitudes[peaks]
sorted_peak_indices = np.argsort(peak_magnitudes)[::-1]
n_highest_peaks = sorted_peak_indices[:3]
highest_freq = frequencies_ext[positive_indices][peaks[n_highest_peaks[0]]]
##


plt.legend()
plt.show()

In [159]:
import numpy as np
from scipy.optimize import minimize

# Parameters
sigma = 5  # Assume some value for sigma
w = 3      # Angular frequency
a = 3    # Damping coefficient

# Function to calculate the determinant of the FIM
def fim_determinant(t, sigma, w, a):
    t0, t1 = t
    I_ww = (t0**2 * np.sin(w*t0)**2 + t1**2 * np.sin(w*t1)**2) * np.exp(-2*a*(t0+t1))
    I_aa = (t0**2 * np.cos(w*t0)**2 + t1**2 * np.cos(w*t1)**2) * np.exp(-2*a*(t0+t1))
    I_wa = (-t0**2 * np.sin(w*t0) * np.cos(w*t0) - t1**2 * np.sin(w*t1) * np.cos(w*t1)) * np.exp(-2*a*(t0+t1))
    determinant = (I_ww * I_aa - I_wa**2) / sigma**4
    return -determinant  # Negative for maximization with minimize()

# Initial guess for t0 and t1
initial_guess = [0.1, 1]

# Constraint to ensure t1 > t0
constraints = ({'type': 'ineq', 'fun': lambda t: t[1] - t[0]})

# Optimization to maximize the determinant of the FIM
result = minimize(fim_determinant, initial_guess, args=(sigma, w, a), constraints=constraints, method='SLSQP')

# Extract the optimal t0 and t1
t0_opt, t1_opt = result.x

print(f"Optimal t0: {t0_opt}, Optimal t1: {t1_opt}")


Optimal t0: 0.1, Optimal t1: 1.0


In [282]:
import random

std = 2
mean = 5
total = 100000
number_of_params = 4
# Generating values and guesses using vectorized operations
values = np.random.normal(mean, std, (total, number_of_params))
guess = np.random.normal(mean, std, (total, number_of_params))
# Calculating MSE using vectorized operations
mse = np.mean((values - guess)**2)
print("mse = ", mse)
print("precentage = ", (np.sqrt(mse)/mean) * 100)


mse =  7.990599270419979
precentage =  56.53529612700363
